# ASL, or American Sign Language
Many deaf people in North America communicate primarily through American Sign Language (ASL), which is also widely used by hard-of-hearing and hearing people.<br>
The development of computer vision systems that translate sign language to spoken language has advanced significantly in recent years. Complex neural network topologies that are capable of identifying minute patterns in streaming video are frequently the basis for this technology. We can lessen the scope of the issue though by translating individual letters rather than entire sentences in order to gain an idea of how to construct a translation system.<br>
In this notebook, we will train a convolutional neural network to identify pictures of letters in ASL. We will train and evaluate the network after loading, preparing and analyzing the data.

## Loading the data

The training and test data are loaded into the code cell below.<br>
x_train and x_test are arrays of image data with shape (num_samples, 3, 50, 50), respectively.<br>
y_train and y_test are arrays of category labels with shape (num_samples,), respectively.<br>


In [ ]:
import numpy as np
np.random.seed(5) 
import tensorflow as tf
tf.set_random_seed(2)
from datasets import sign_language
import matplotlib.pyplot as plt
%matplotlib inline

(x_train, y_train), (x_test, y_test) = sign_language.load_data()

## Visualizing the training data
To start, we will create a list of labels containing the letters appearing in the dataset, then we visualize the first entries in the training data.

In [ ]:
labels=['A','B','C']
fig=plt.figure(figsize=(20,5))
for i in range(36):
    ax=fig.add_subplot(3,12,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(x_train[i]))
    ax.set_title("{}".format(labels[y_train[i]]))
plt.show()

## Verifying the dataset integrity

The datasets x_train and x_test contain the images, while y_train and y_test contain the corresponding letters.<br>
The values for each element in y_train and y_test are 0, 1, or 2, which stand for the letters "A," "B," and "C," respectively.<br>
To confirm that the training and test sets contain approximately equal amounts of each letter, we will use the arrays y_train and y_test.

In [ ]:
num_A_train = sum(y_train==0)
num_B_train = sum(y_train==1)
num_C_train = sum(y_train==2)
num_A_test = sum(y_test==0)
num_B_test = sum(y_test==1)
num_C_test = sum(y_test==2)
print("Training set:")
print("\tA: {}, B: {}, C: {}".format(num_A_train, num_B_train, num_C_train))
print("Test set:")
print("\tA: {}, B: {}, C: {}".format(num_A_test, num_B_test, num_C_test))

## One-hot encoding
At the moment, we have category integer labels for each letter; for example, the labels for "A," "B," and "C" are encoded as 0, 1, and 2, respectively. But keep in mind that this format is not supported by Keras models, thus before feeding the labels to a Keras model, we need to one-hot encode them.<br>
The one-dimensional label array will become a two-dimensional array as a result of this conversion.<br>
<br>
A distinct image is represented by each row in the two-dimensional array of one-hot encoded labels. The row has a 0 in other columns and a 1 in the column that matches the correct label.<br>
As an example, the codes for 0 and 1 are [1, 0, 0], [0, 1, 0], and [0, 0, 1], respectively.

In [ ]:
y_train_OH=np_utils.to_categorical(y_train)
y_test_OH=np_utils.to_categorical(y_test)
model=Sequential()
model.add(Conv2D(filters=5,kernel_size=5,padding='same',activation='relu',input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=4))
model.add(Conv2D(filters=15,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=4))
model.add(Flatten())
model.add(Dense(3,activation='softmax'))
model.summary()


## Compiling the model

After one-hot encoding the data, we can proceed with feeding it to the Keras model and compile the neural network.

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

## Training the model

We fit the model to the training data.

In [ ]:
hist=model.fit(x_train, y_train_OH,
               validation_split=0.2,
               epochs=2, batch_size=32)

## Testing the model

We will use the test dataset to evaluate the model. This will provide insight into the network's performance in categorizing previously unseen images!

It is a positive indication that the model did not overfit to the training set if the test dataset's classification accuracy is comparable to that of the training dataset.

In [ ]:
score=model.evaluate(x_test,y_test_OH,verbose=0)
print('Test accuracy:', score[1])

## Visualizing mistakes
On the test set, our network achieves really good accuracy.

Reviewing the photographs can occasionally reveal unique features that the model finds puzzling. But it also frequently happens that it's difficult to understand what the model intended.

In [ ]:
y_probs = model.predict(x_test)
y_preds = np.argmax(y_probs, axis=1)
bad_test_idxs = np.where(y_preds != y_test)[0]
fig=plt.figure(figsize=(25,4))
for i, idx in enumerate(bad_test_idxs):
    ax = fig.add_subplot(2, np.ceil(len(bad_test_idxs)/2), i+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    ax.set_title("{} (pred: {})".format(labels[y_test[idx]], labels[y_preds[idx]]))